# Imports

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torch.amp import autocast, GradScaler

import numpy as np
import matplotlib.pyplot as plt
import time
import os
from tqdm import tqdm

In [12]:
torch.manual_seed(42)
np.random.seed(42)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# Constants

In [ ]:
IMAGE_SIZE = 32
NUM_CLASSES = 100
CIFAR10_MEAN = [0.5071, 0.4867, 0.4408]
CIFAR10_STD = [0.2675, 0.2565, 0.2761]

BATCH_SIZE = 128
TEACHER_LR = 0.001
RANDOM_INIT_LR = 0.001
STUDENT_DISTILL_LR = 0.001
WEIGHT_DECAY = 0.01

# Load Data

In [15]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                             download=True, transform=train_transform)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                            download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"Training dataset size: {len(train_dataset)}")
print(f"Testing dataset size: {len(test_dataset)}")
print(f"Number of classes: {len(train_dataset.classes)}")

Files already downloaded and verified
Files already downloaded and verified
Training dataset size: 50000
Testing dataset size: 10000
Number of classes: 100


# Models (Teacher and Student)

In [16]:
def create_teacher_model():
    model = models.resnet50(weights=None)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 100)
    
    return model

def create_student_model():
    model = models.mobilenet_v2(weights=None)
    model.features[0][0] = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
    model.classifier = nn.Linear(model.last_channel, 100)

    return model

In [17]:
teacher_model = create_teacher_model().to(device)
student_model = create_student_model().to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Teacher model (ResNet-50) parameters: {count_parameters(teacher_model):,}")
print(f"Student model (ResNet-18) parameters: {count_parameters(student_model):,}")

Teacher model (ResNet-50) parameters: 23,705,252
Student model (ResNet-18) parameters: 2,351,972


# Training the Teacher Model

In [22]:
def train_model(model, train_loader, test_loader, epochs, device, save_path=None):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    scaler = GradScaler()
    
    best_acc = 0.0
    
    history = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': []
    }
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for inputs, targets in progress_bar:
            inputs = inputs.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            
            optimizer.zero_grad()
            
            with autocast(device_type=device.type):
                logits = model(inputs)
                loss = criterion(logits, targets)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = logits.max(1)
            train_total += targets.size(0)
            train_correct += predicted.eq(targets).sum().item()

            progress_bar.set_postfix(
                {"loss": loss.item(), "acc": 100 * train_correct / train_total}
            )
        
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = 100.0 * train_correct / train_total
        
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0
        progress_bar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Test]")
        with torch.no_grad():
            for inputs, targets in progress_bar:
                inputs = inputs.to(device, non_blocking=True)
                targets = targets.to(device, non_blocking=True)
                
                logits = model(inputs)
                loss = criterion(logits, targets)
                
                test_loss += loss.item() * inputs.size(0)
                _, predicted = logits.max(1)
                test_total += targets.size(0)
                test_correct += predicted.eq(targets).sum().item()

                progress_bar.set_postfix(
                    {"loss": loss.item(), "acc": 100 * test_correct / test_total}
                )
        
        test_loss = test_loss / len(test_loader.dataset)
        test_acc = 100.0 * test_correct / test_total

        scheduler.step()
        
        if test_acc > best_acc and save_path:
            best_acc = test_acc
            torch.save(model.state_dict(), save_path)
            print(f"Saved best model with accuracy: {best_acc:.2f}%")
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        
        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    
    return model, history

In [23]:
os.remove('/kaggle/working/teacher_model.pth')

In [ ]:
teacher_path = './teacher_model.pth'
if os.path.exists(teacher_path):
    print("Loading pre-trained teacher model...")
    teacher_model.load_state_dict(torch.load(teacher_path, weights_only=True))
else:
    print("Training teacher model...")
    teacher_model, teacher_history = train_model(
        teacher_model, train_loader, test_loader, epochs=100, device=device, save_path=teacher_path
    )

Training teacher model...


Epoch 1/100 [Test]: 100%|██████████| 79/79 [00:05<00:00, 14.48it/s, loss=3.84, acc=10.5]


Saved best model with accuracy: 10.54%
Epoch 1/100 - Train Loss: 4.1612, Train Acc: 7.12%, Test Loss: 3.8179, Test Acc: 10.54%


Epoch 2/100 [Test]: 100%|██████████| 79/79 [00:05<00:00, 14.54it/s, loss=3.1, acc=18.4] 


Saved best model with accuracy: 18.39%
Epoch 2/100 - Train Loss: 3.6806, Train Acc: 13.29%, Test Loss: 3.4289, Test Acc: 18.39%


...

Epoch 86/100 [Test]: 100%|██████████| 79/79 [00:05<00:00, 14.64it/s, loss=0.752, acc=72.1]


Saved best model with accuracy: 72.09%
Epoch 86/100 - Train Loss: 0.3902, Train Acc: 89.25%, Test Loss: 1.4990, Test Acc: 72.09%


Epoch 87/100 [Test]: 100%|██████████| 79/79 [00:05<00:00, 14.62it/s, loss=0.851, acc=72.2]


Saved best model with accuracy: 72.19%
Epoch 87/100 - Train Loss: 0.3863, Train Acc: 89.55%, Test Loss: 1.4954, Test Acc: 72.19%


Epoch 88/100 [Test]:   6%|▋         | 5/79 [00:00<00:05, 12.36it/s, loss=1.59, acc=73.1]

# Knowledge Distillation

In [ ]:
class DistillationLoss(nn.Module):
    def __init__(self, alpha=0.5, temperature=4.0):
        super(DistillationLoss, self).__init__()
        self.alpha = alpha
        self.temperature = temperature
        self.criterion = nn.CrossEntropyLoss()
    
    def forward(self, student_outputs, teacher_outputs, targets):
        hard_loss = self.criterion(student_outputs, targets)
        soft_student = F.log_softmax(student_outputs / self.temperature, dim=1)
        soft_teacher = F.softmax(teacher_outputs / self.temperature, dim=1)
        soft_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (self.temperature ** 2)
        return (1 - self.alpha) * hard_loss + self.alpha * soft_loss

# Training Students (From Scratch vs Distillation)

In [ ]:
student_model_baseline = create_student_model().to(device)

print("Training student model from scratch without distillation...")
student_baseline_path = './student_baseline_model.pth'
student_model_baseline, student_baseline_history = train_model(
    student_model_baseline, train_loader, test_loader, epochs=100,
    save_path=student_baseline_path, is_teacher=False
)

In [ ]:
def train_student_with_distillation(student_model, teacher_model, train_loader, test_loader, epochs, alpha=0.5, temperature=4.0, save_path=None):
    distillation_criterion = DistillationLoss(alpha=alpha, temperature=temperature)
    standard_criterion = nn.CrossEntropyLoss()
    
    optimizer = optim.AdamW(student_model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    scaler = GradScaler()
    
    best_acc = 0.0
    
    history = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': []
    }
    
    for epoch in range(epochs):
        student_model.train()
        teacher_model.eval()
        
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for inputs, targets in progress_bar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            
            with autocast():
                student_outputs = student_model(inputs)
                with torch.no_grad():
                    teacher_outputs = teacher_model(inputs)
                
                loss = distillation_criterion(student_outputs, teacher_outputs, targets)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item() * inputs.size(0)
            _, predicted = student_outputs.max(1)
            train_total += targets.size(0)
            train_correct += predicted.eq(targets).sum().item()

            progress_bar.set_postfix(
                {"loss": loss.item(), "acc": 100 * train_correct / train_total}
            )
        
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = 100.0 * train_correct / train_total

        student_model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0
        progress_bar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Valid]")
        with torch.no_grad():
            for inputs, targets in progress_bar:
                inputs, targets = inputs.to(device), targets.to(device)
                
                outputs = student_model(inputs)
                loss = standard_criterion(outputs, targets)
                
                test_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                test_total += targets.size(0)
                test_correct += predicted.eq(targets).sum().item()

                progress_bar.set_postfix(
                    {"loss": loss.item(), "acc": 100 * test_correct / test_total}
                )
        
        test_loss = test_loss / len(test_loader.dataset)
        test_acc = 100.0 * test_correct / test_total
        
        scheduler.step()
        
        if test_acc > best_acc and save_path:
            best_acc = test_acc
            torch.save(student_model.state_dict(), save_path)
            print(f"Saved best model with accuracy: {best_acc:.2f}%")
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        
        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
              f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    
    return student_model, history

print("Training student model with knowledge distillation...")
student_distill_path = './student_distill_model.pth'
student_model_distill, student_distill_history = train_student_with_distillation(
    student_model, teacher_model, train_loader, test_loader, 
    epochs=100, alpha=0.5, temperature=4.0,
    save_path=student_distill_path
)

# Evaluation and Comparison

In [ ]:
def evaluate_model(model, test_loader, model_name):
    model.eval()
    correct = 0
    total = 0
    
    class_correct = list(0. for i in range(100))
    class_total = list(0. for i in range(100))
    
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc=f"Evaluating {model_name}"):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            c = (predicted == targets).squeeze()
            for i in range(targets.size(0)):
                label = targets[i].item()
                class_correct[label] += c[i].item()
                class_total[label] += 1
    
    overall_acc = 100.0 * correct / total
    print(f"{model_name} - Test Accuracy: {overall_acc:.2f}%")
    
    class_accuracies = []
    for i in range(100):
        if class_total[i] > 0:
            class_acc = 100.0 * class_correct[i] / class_total[i]
            class_accuracies.append(class_acc)
    
    avg_class_acc = sum(class_accuracies) / len(class_accuracies)
    print(f"{model_name} - Average Class Accuracy: {avg_class_acc:.2f}%")
    
    return overall_acc, avg_class_acc

teacher_model.load_state_dict(torch.load(teacher_path))
student_model_baseline.load_state_dict(torch.load(student_baseline_path))
student_model_distill.load_state_dict(torch.load(student_distill_path))

teacher_acc, teacher_class_acc = evaluate_model(teacher_model, test_loader, "Teacher (ResNet-50)")
baseline_acc, baseline_class_acc = evaluate_model(student_model_baseline, test_loader, "Student Baseline (ResNet-18)")
distill_acc, distill_class_acc = evaluate_model(student_model_distill, test_loader, "Student with Distillation (ResNet-18)")

# Visualization and Analysis

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(student_baseline_history['test_acc'], label='Student Baseline')
plt.plot(student_distill_history['test_acc'], label='Student with Distillation')
plt.axhline(y=teacher_acc, color='r', linestyle='--', label='Teacher')
plt.title('Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(student_baseline_history['test_loss'], label='Student Baseline')
plt.plot(student_distill_history['test_loss'], label='Student with Distillation')
plt.title('Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

models = ['Teacher (ResNet-50)', 'Student Baseline (ResNet-18)', 'Student with Distillation (ResNet-18)']
accuracies = [teacher_acc, baseline_acc, distill_acc]
model_sizes = [count_parameters(teacher_model), count_parameters(student_model_baseline), count_parameters(student_model_distill)]

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.bar(models, accuracies, color=['blue', 'orange', 'green'])
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy (%)')
plt.xticks(rotation=15, ha='right')
plt.ylim(0, 100)
for i, v in enumerate(accuracies):
    plt.text(i, v + 1, f"{v:.2f}%", ha='center')

plt.subplot(1, 2, 2)
sizes_in_millions = [s / 1000000 for s in model_sizes]
plt.bar(models, sizes_in_millions, color=['blue', 'orange', 'green'])
plt.title('Model Size Comparison')
plt.ylabel('Parameters (millions)')
plt.xticks(rotation=15, ha='right')
for i, v in enumerate(sizes_in_millions):
    plt.text(i, v + 0.1, f"{v:.2f}M", ha='center')

plt.tight_layout()
plt.show()

# Inference Speed Comparison

In [ ]:
def measure_inference_time(model, input_size=(128, 3, 32, 32), iterations=100):
    model.eval()
    x = torch.randn(input_size).to(device)
    
    # Warm-up
    with torch.no_grad():
        for _ in range(10):
            _ = model(x)
    
    # Measure
    torch.cuda.synchronize()
    start_time = time.time()
    
    with torch.no_grad():
        for _ in range(iterations):
            _ = model(x)
    
    torch.cuda.synchronize()
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    return elapsed_time / iterations * 1000  # Convert to ms per batch

teacher_time = measure_inference_time(teacher_model)
student_baseline_time = measure_inference_time(student_model_baseline)
student_distill_time = measure_inference_time(student_model_distill)

print(f"Teacher (ResNet-50) inference time: {teacher_time:.2f} ms/batch")
print(f"Student Baseline (ResNet-18) inference time: {student_baseline_time:.2f} ms/batch")
print(f"Student with Distillation (ResNet-18) inference time: {student_distill_time:.2f} ms/batch")
print(f"Speed-up: {teacher_time / student_distill_time:.2f}x")

plt.figure(figsize=(8, 6))
inference_times = [teacher_time, student_baseline_time, student_distill_time]
plt.bar(models, inference_times, color=['blue', 'orange', 'green'])
plt.title('Inference Time Comparison')
plt.ylabel('Time per batch (ms)')
plt.xticks(rotation=15, ha='right')
for i, v in enumerate(inference_times):
    plt.text(i, v + 0.2, f"{v:.2f} ms", ha='center')
plt.tight_layout()
plt.show()